In [11]:
import pymysql
import pandas as pd
import csv

# We want to fill in the missing product IDs after encountering errors during csv imports to sql

In [8]:
import chardet

with open('../Data/Superstore.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))  # check first 10k bytes
print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [7]:
supastore = pd.read_csv('../Data/superstore.csv', encoding='ISO-8859-1')

In [8]:
pid_unique = supastore.drop_duplicates(subset='Product ID')

In [2]:
# Connection config
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "root",
    "database": "supastore(2025)"
}

In [3]:
# Connect to the database
conn = pymysql.connect(**db_config)
cursor = conn.cursor()

In [12]:
#Get existing Product IDs from the target table
cursor.execute("SELECT `ï»¿Product ID` FROM products")
existing_ids = {row[0] for row in cursor.fetchall()}

In [13]:
#Filter only missing Product IDs
df_missing = pid_unique[~pid_unique['Product ID'].isin(existing_ids)]

In [21]:
df_missing['Product ID'].value_counts().sum()

np.int64(1764)

In [15]:
# Insert missing rows into the database
insert_query = """
    INSERT INTO products (`ï»¿Product ID`, `Product Name`, `Category`, `Sub-Category`)
    VALUES (%s, %s, %s, %s)
"""
cursor.executemany(insert_query, df_missing[['Product ID', 'Product Name', 'Category', 'Sub-Category']].values.tolist())
conn.commit()

print(f"Inserted {len(df_missing)} missing unique product rows.")

# Cleanup
cursor.close()
conn.close()

Inserted 1764 missing unique product rows.


In [13]:
cursor.execute("SELECT * FROM products")
prod =  cursor.fetchall()

In [14]:
columns = [desc[0] for desc in cursor.description]

In [18]:
with open("C:\supastore\Data\clean_products.csv", "w", newline='', encoding = "utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(columns)
    writer.writerows(prod)